In [1]:
import numpy as np
import yfinance as yf
import datetime
import pandas as pd

In [2]:
def download_data(stock, start, end):
    data = {}
    ticker = yf.download(stock, start, end)
    data['Adj Close'] = ticker['Adj Close']
    return pd.DataFrame(data)

In [3]:
class ValueAtRiskMonteCarlo:

    def __init__(self, S, mu, sigma, c, n, iterations):
        self.S = S
        self.mu = mu
        self.sigma = sigma
        self.c = c
        self.n = n
        self.iterations = iterations

    def simulation(self):
        stock_data = np.zeros([self.iterations, 1])
        rand = np.random.normal(0, 1, [1, self.iterations])

        # equation for the S(t) stock price
        stock_price = self.S * np.exp(self.n * (self.mu - 0.5 * self.sigma ** 2) + self.sigma * np.sqrt(self.n) * rand)

        # we have to sort the stock prices to determine the percentile
        stock_price = np.sort(stock_price)

        # it depends on the confidence level: 95% -> 5 and 99% -> 1
        percentile = np.percentile(stock_price, (1 - self.c) * 100)

        return self.S - percentile

In [4]:
S = 1e6  # this is the investment (stocks or whatever)
c = 0.95  # condifence level: this time it is 99%
n = 1  # 1 day
iterations = 1000000  # number of paths in the Monte-Carlo simulation

# historical data to approximate mean and standard deviation
start_date = datetime.datetime(2019, 1, 1)
end_date = datetime.datetime(2022, 1, 1)

In [5]:
# download stock related data from Yahoo Finance
tesla = download_data('TSLA', start_date, end_date)

# calculate daily returns
tesla['returns'] = np.log(tesla['Adj Close']/tesla['Adj Close'].shift(1))

[*********************100%***********************]  1 of 1 completed


In [6]:
tesla = tesla[1:]

In [7]:
# we can assume daily returns to be normally sidtributed: mean and variance (standard deviation)
# can describe the process
mu = np.mean(tesla['returns'])
sigma = np.std(tesla['returns'])

model = ValueAtRiskMonteCarlo(S, mu, sigma, c, n, iterations)

print('Value at risk with Monte-Carlo simulation: $%0.2f' % model.simulation())

Value at risk with Monte-Carlo simulation: $64235.52
